# STAR-HTSeq-DEseq2 pipeline for RNAseq analysis

### dataset: 5 paired-end RNAseq 
             2 control and 3 experiment samples
             sequence depth: ~50M reads/sample

### STAR alignment: 15-20 minutes/full dataset (8 threads)
### HTseq: ~2-3 hours/full dataset (1 thread)   ~5 minutes/test files of 400K SAM reads       
### DEseq2: 

# STAR

## genome index
__1. generate your own genome index(recommended)

   a. download the genome fasta files
   b. run STAR in genome index mode
   e.g. 
   STAR --runThreadN 4 --runMode genomeGenerate --genomeDir Genome_data/star \
   --genomeFastaFiles Genome_data/Mus_musculus_UCSC_mm10.tar.gz
   
   
__2. download from Star genome (human and mouse)

   link:http://labshare.cshl.edu/shares/gingeraslab/www-data/dobin/STAR/STARgenomes/
   
   star genome index contains: {chrLength.txt  chrName.txt      chrStart.txt    sjdbInfo.txt    Genome  
    genomeParameters.txt   SA    SAindex}

## STAR command and options
   
   STAR --option1-name option1-value(s) --option2-name option2-value(s)....
   
   _1. required
    --runThreadN NumberOfThreads
    --genomeDir /path/to/genomeDir
    --readFilesIn /path/to/read1 [/path/to/read2]
    --outFileNamePrefix /path/to/output/dir/prefix
    
   _2. Optional
    --readFilesCommand UncompressionCommand e.g.zcat - to uncompress .gz files
    --outSAMmapqUnique Integer0to255 e.g. mapping quality MAPQ=255 for the unique mappers
    --outSAMtype SAM/BAM Unsorted/SortedByCoordinate

In [ ]:
# extract path information for raw fastq and output from config file and store them in lists
file_read = open("STAR_align.config", "r")
fastq_list = []
output_list = []

for line in file_read:
    if line.startswith("RAWfastq>"):
        line=line.strip()
        line=line[9:]
        fastq_list.append(line)
    if line.startswith("output>"):
        line=line.strip()
        line=line[7:]
        output_list.append(line)
        
print(fastq_list)
print(output_list)
file_read.close()

## how to pass variables from python to bash: use $

In [ ]:
# use for loop to go through all the files in STAR and store the output files in respective directories.
dataset_count =0
for index in range (0,len(fastq_list),2):
    read_1 = fastq_list[index]
    read_2 = fastq_list[index+1]
    output_path = output_list[dataset_count]
    dataset_count += 1
    # print(read_1, read_2, output_path)
    ! time STAR --runThreadN 8 --genomeDir /data/project/refgenome_STAR  --readFilesIn $read_1 $read_2 \
    --outSAMmapqUnique 255 -- outSAMtype BAM SortedByCoordinate \
    --outFileNamePrefix $output_path
    # break

# HTseq

   
   ## 1. gene annotation - GTF or GFF file for HTseq count funciton
         e.g. human Gencode V25 - comprehansive gene annotations
         link: http://www.gencodegenes.org/releases/25.html
   
   ## 2. sorted BAM files from STAR aligner
         note: STAR sort BAM based on mapped read coordinates
         
   ## 3. htseq-count: 
         more information: http://htseq.readthedocs.io/en/release_0.9.1/count.html?highlight=htseq-count
   
         Basic command line: htseq-count [options] <alignment_files> <gff_file> > htseq_output_countfile
         options:
         -f <format>, --format=<format> sam/bam
         -r <order>, --order=<order> name/pros (aligned read ID/positions)
                     for paired end reads, alignment files have to be sorted.
         
         -i <id attribute>, --idattr=<id attribute> e.g. gene_id, gene_name

In [ ]:
# extract path information for raw fastq and output from config file and store them in lists

file_read = open("HTseq.config", "r")
input_dir = []
output_dir = ""
output_name = []
output_group = []

for line in file_read:
    if line.startswith("inputBAM>"):  #parse path for BAM input file
        line = line[9:]
        line = line.strip()
        input_dir.append(line)
    if line.startswith("outputpath>"): #parse path for output HTseq files
        line=line.strip()
        output_dir = line[11:]
    if line.startswith("groupinfo>"):  #parse dataset info (dataID and group/condition)
        line = line[10:]
        output = line.strip().split("_")
        output_name.append(output[0])
        output_group.append(output[1])
#print(input_dir)
#print(output_dir)
#print(output_name)
#print(output_group)
file_read.close()


file_write = open ("/data/project/HTseq/deseq2_inputsummary.txt", "w") # initiate a meta file table for downstream DESeq
file_write.write("sampleID\t"+"filenames\t"+"conditions"+"\n")

for i in range(0,len(input_dir)):
    input_path = input_dir[i]
    output_filename = output_name[i]+"_"+output_group[i]+"_output.txt"
    output_path_name = str(output_dir)+output_filename
    #print(input_path)
    #print(output_filename)
    #print(output_path_name)
    file_write.write(output_name[i]+"\t"+output_filename+"\t"+output_group[i]+"\n")
    
    ! time htseq-count -f bam -r pos -i gene_name \
    $input_path GencodeV25_GTF/CHR_comprehansive_V25.gtf > $output_path_name

file_write.close()
    

# DESeq2

In [ ]:
! Rscript /data/project/HTseq/DEseq2_test2.r

DEseq2_test2.r


meta <- read.table("/data/project/HTseq/htseq_test2/deseq2_inputsummary.txt", header =TRUE)

#this summary table was generated from HTSeq run and contain sampleID, HTSeq count output file names and experimental conditions of your datasets.
#Note: the directory of these files is assigned specifically as DEseqDataSetFromHTSeqCount variable. Don't put path info here in the meta table.


#assign attributes of sampleTable based on meta table

samplenames <- as.character(meta$sampleID)
filenames <- as.character(meta$filenames)
samplecondition <- as.character(meta$condition)

sampleTable <- data.frame(sampleName = samplenames, fileName = filenames, condition = samplecondition)
print(sampleTable)


library(DESeq2) # Call DESeq2 in R and input HTseq count

directory <- "/data/project/HTseq/htseq_test2/" # assign directory of HTseq count output files


# assign the sample table to DESeq function for HTseq count input

ddsHTSeq <- DESeqDataSetFromHTSeqCount (sampleTable = sampleTable, directory = directory, design = ~ condition)
print(ddsHTSeq)

dds <- DESeq(ddsHTSeq)
res <- results(dds)  
res05 <- results(dds,alpha=0.05) # highlight DE genes with FDR <0.05
summary(res05) 
sum(res05$padj <0.05, na.rm=TRUE) # how many DE genes meet the threshhold

plotMA(res05, ylim=c(-10,10)) # MAplot
jpeg("DEseq_res05_MAplot2.jpg")
plotMA(res05, ylim=c(-10,10))
dev.off()

res05Ordered <- res05[order(res05$pvalue),] 
write.csv(as.data.frame(res05Ordered), file="parental_brain_FDR05_results.csv") # export results into csv fles

  sampleName                 fileName condition
1 SRR3649333 htseq_test333_output.txt  parental
2 SRR3649334 htseq_test334_output.txt  parental
3 SRR3649335 htseq_test335_output.txt     brain
4 SRR3649336 htseq_test336_output.txt     brain
5 SRR3649337 htseq_test337_output.txt     brain
Loading required package: S4Vectors
Loading required package: methods
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, as.vector, cbind, colnames,
    do.call, duplicated, eval, evalq, Filter, Find, get, grep, grepl,
    intersect, is.unsorted, lapply, lengths, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unlist, unsplit

Loading required package: IRanges
Loading required package: GenomicRanges
Loading required package: GenomeInfoDb
Loading required package: SummarizedExperiment
Loading required package: Biobase
Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")', and for packages 'citation("pkgname")'.

Loading required package: Rcpp
Loading required package: RcppArmadillo
class: DESeqDataSet 
dim: 56269 5 
metadata(0):
assays(1): counts
rownames(56269): 5S_rRNA 5_8S_rRNA ... uc_338 yR211F11.2
rowRanges metadata column names(0):
colnames(5): SRR3649333 SRR3649334 SRR3649335 SRR3649336 SRR3649337
colData names(1): condition
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing

out of 95 with nonzero total read count
adjusted p-value < 0.05
LFC > 0 (up)     : 6, 6.3% 
LFC < 0 (down)   : 0, 0% 
outliers [1]     : 0, 0% 
low counts [2]   : 0, 0% 
(mean count < 0)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

[1] 6
pdf 
  2 